# Configuração do Google Colab para Treinamento Distribuído Relex.AI

Este notebook demonstra como configurar o ambiente Google Colab como nó de trabalho para o sistema de treinamento distribuído Relex.AI.

## 1. Verificando Ambiente Colab

Primeiro, vamos verificar os recursos disponíveis neste ambiente Colab.

In [ ]:
# Verificar tipo de GPU disponível
!nvidia-smi

# Verificar informações do sistema
!cat /proc/cpuinfo | grep "model name" | head -1
!free -h
!df -h /content

## 2. Download do Script de Configuração

Agora vamos baixar o script de configuração para o Google Colab.

In [ ]:
# Criar diretório para o script, se não existir
!mkdir -p scripts

# Baixar o script de configuração do repositório
!wget -O scripts/setup_colab.py https://raw.githubusercontent.com/relex-ai/distributed-training/main/scripts/setup_colab.py

# Verificar se o download foi bem-sucedido
!ls -la scripts/

## 3. Configurar Token ngrok

Precisamos configurar um token do ngrok para criar um túnel que permita comunicação externa com o nó de trabalho.

In [ ]:
# Configure seu token ngrok aqui
# Você pode obter um token gratuito em https://dashboard.ngrok.com/get-started/your-authtoken
NGROK_TOKEN = "seu_token_ngrok_aqui"

## 4. Definir Parâmetros de Configuração

Configure os parâmetros necessários para conectar ao servidor mestre.

In [ ]:
# Configurações do servidor mestre
MASTER_IP = "seu_master_ip_aqui"  # Substitua pelo IP do servidor mestre
MASTER_PORT = 5000  # Porta padrão, ajuste se necessário

# Token de autenticação para registro com o servidor mestre
AUTH_TOKEN = "seu_token_de_autenticacao_aqui"

# Configurações adicionais (opcionais)
WORKER_PORT = 5001  # Porta local do nó de trabalho
WORKING_DIR = "/content/relex"  # Diretório de trabalho
NODE_NAME = "colab-worker-1"  # Nome personalizado para este nó

## 5. Executar o Script de Configuração

Agora vamos executar o script com os parâmetros definidos.

In [ ]:
!python scripts/setup_colab.py \
    --master-ip $MASTER_IP \
    --master-port $MASTER_PORT \
    --worker-port $WORKER_PORT \
    --working-dir $WORKING_DIR \
    --token $AUTH_TOKEN \
    --ngrok-token $NGROK_TOKEN \
    --node-name $NODE_NAME

## 6. Monitorar o Nó de Trabalho

Uma vez configurado, o script iniciará automaticamente o processo de worker. Você pode monitorar os logs para verificar o progresso.

In [ ]:
# Verificar logs para monitorar o progresso
!tail -f $WORKING_DIR/logs/worker.log

## 7. Verificar Status

Você pode verificar o status do nó de trabalho e sua conexão com o servidor mestre.

In [ ]:
# Verificar se o processo worker está em execução
!ps aux | grep run_worker

# Verificar configuração do worker
!cat $WORKING_DIR/worker_config.json

## 8. Manter Colab Ativo

Para evitar que o Google Colab entre em modo de inatividade e desconecte o worker, você pode executar o seguinte código:

In [ ]:
# Função para manter o Colab ativo
import time
import IPython.display
from IPython.display import HTML, display

def keep_alive():
    display(HTML('''
    <script>
        function ClickConnect(){
            console.log("Mantendo conexão ativa");
            document.querySelector("colab-connect-button").click()
        }
        setInterval(ClickConnect, 60000)
    </script>
    ''')
    )
    
    while True:
        time.sleep(60)
        IPython.display.clear_output(wait=True)
        print(f"Sessão Colab mantida ativa - Último ping: {time.strftime('%Y-%m-%d %H:%M:%S')}")

# Descomente a linha abaixo para iniciar o script keep-alive
# keep_alive()

---

**Nota:** Este notebook deve ser mantido em execução enquanto você deseja que este ambiente Colab participe do treinamento distribuído. O fechamento do notebook ou a expiração da sessão desconectará o nó do sistema distribuído.